### Load all Dataframes

In [ ]:
import pandas as pd

# Read the CSV files into dataframes
policy_assignments_df = pd.read_csv('../output/policyAssignments.csv')
export_users_df = pd.read_csv('../output/exportUsers_2025-1-23.csv')
policy_definitions_df = pd.read_csv('../output/policyDefinitions.csv')
initiave_policies_df = pd.read_csv('../output/initiativePolicies.csv')
management_groups_df = pd.read_csv('../output/managementGroups.csv')

# Display the dataframes
# print(policy_assignments_df.head())
# print(export_users_df.head())
# policy_assignments_df.head()


In [ ]:
# merge with ManagementGroup DF - "Scope" column from initiave_policies_df and "id" column from management_groups_df. Only retrieve columns "properties_displayName" and "mgParent_displayName"
policy_assignments_analysis_df = pd.merge(policy_assignments_df, management_groups_df[['id', 'properties_displayName', 'parentDisplayName']], left_on='Scope', right_on='id', how='left')
policy_assignments_analysis_df.drop('id', axis=1, inplace=True)

# Merge the dataframes adding user to the Policy Assignment sheet. The column ID of the Users sheet is the same as the column CreatedBy of the Policy Assignment sheet
policy_assignments_analysis_df = pd.merge(policy_assignments_analysis_df, export_users_df[['id', 'displayName']], left_on='createdBy', right_on='id', how='left')
policy_assignments_analysis_df.drop(['id','createdBy'], axis=1, inplace=True)

# Merge with policy_definitions_df - "Policy Name" column with column "Name" from policy_definitions_df. Only retrieve the columns "Category" "AvailableEffects"	"Description"
policy_assignments_analysis_df = pd.merge(policy_assignments_analysis_df, policy_definitions_df[['Id','Category', 'AvailableEffects', 'Description']], left_on='policyDefinitionId', right_on='Id', how='left')
policy_assignments_analysis_df.drop(['Id'], axis=1, inplace=True)

# policy_assignments_analysis_df has 1:N relationship with initiave_policies_df. Merge with initiave_policies_df - "InitiativeId" column with column "Policy Name" from policy_assignments_analysis_df. Transform all the items of initiative_policies_df into a list and add it to the policy_assignments_analysis_df
# Group initiave_policies_df by 'InitiativeId' and aggregate the rows into lists
initiative_policies_grouped = initiave_policies_df[['InitiativeId', 'PolicyName']].groupby('InitiativeId').agg(lambda x: '\n'.join(x)).reset_index()

# Merge the grouped initiative_policies_df with policy_assignments_analysis_df
policy_assignments_analysis_df = pd.merge(policy_assignments_analysis_df, initiative_policies_grouped, left_on='policyDefinitionId', right_on='InitiativeId', how='left')

# Drop the 'InitiativeId' column as it is redundant after the merge
policy_assignments_analysis_df.drop(['InitiativeId','policyDefinitionId'], axis=1, inplace=True)

# Rename the "displayName" column to "createdByDisplayName"
policy_assignments_analysis_df.rename(columns={'displayName': 'createdByDisplayName', 'Category': 'Policy Category', 'Description': 'Policy Description', 'Type':'Policy Type', 'PolicyName': 'Initiative Policies', 'properties_displayName': 'Mgmt Group'}, inplace=True)


# Sort the dataframe by "Scope Type"
policy_assignments_analysis_df.sort_values(by='Scope Type', inplace=True)

# Make sure that policy_assignments_analysis_df lenght is the same 
assert len(policy_assignments_analysis_df) == len(policy_assignments_df), f"The length of policy_assignments_analysis_df ({len(policy_assignments_analysis_df)}) does not match the length of policy_assignments_df ({len(policy_assignments_df)})"

# policy_assignments_analysis_df.head()

### Generate the AEP final sheet

In [ ]:
# Filter the rows where the "Scope Name" contains "AEP"
policy_assignments_analysis_aep_df = policy_assignments_analysis_df[policy_assignments_analysis_df['Scope Name'].str.contains('AEP', case=False)].copy()

# Column "Scope Name" = if "Scope Type" is "Management Group" then "Mgmt Group" else "Scope Name"
policy_assignments_analysis_aep_df.loc[:, 'Scope Name'] = policy_assignments_analysis_aep_df.apply(lambda x: x['Mgmt Group'] if x['Scope Type'] == 'Management Group' else x['Scope Name'], axis=1)

# Only keep columns "Scope Type", "Scope Name", "Kind", "Policy Category", "Policy Type", "Policy Name" and "Initiative Policies"
policy_assignments_analysis_aep_df = policy_assignments_analysis_aep_df[['Scope Type', 'Scope Name', 'Kind', 'Policy Category', 'Policy Type', 'Policy Name', 'Initiative Policies']]

# Sort the dataframe by "Scope Type"
policy_assignments_analysis_aep_df.sort_values(by='Scope Type', inplace=True)

# policy_assignments_analysis_aep_df.head()


### Write to Excel

In [22]:
with pd.ExcelWriter('../output/policyAssignmentsAnalysis.xlsx') as writer:
    policy_assignments_analysis_aep_df.to_excel(writer, sheet_name='AEP Assignments Analysis', index=False)
    policy_assignments_analysis_df.to_excel(writer, sheet_name='Policy Assignments', index=False)
    policy_definitions_df.to_excel(writer, sheet_name='Policy Definitions', index=False)
    initiave_policies_df.to_excel(writer, sheet_name='Initiative Policies', index=False)
    management_groups_df.to_excel(writer, sheet_name='Management Groups', index=False)